In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Yolo/Face_Mask_detection

/content/drive/MyDrive/Yolo/Face_Mask_detection


Prepare Dataset

In [ ]:
import json
import zipfile
import os

# api key will be hidden due to privacy 
api_token = {"username":"wongguansheng","key":"4fd69250da8b40510dc2df442667ec09"}

if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")
 
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
! chmod 600 /root/.kaggle/kaggle.json

if not os.path.exists("/content/drive/MyDrive/Yolo/Face_Mask_detection/Face_Mask_data"):
    os.makedirs("/content/drive/MyDrive/Yolo/Face_Mask_detection/Face_Mask_data")
os.chdir('/content/drive/MyDrive/Yolo/Face_Mask_detection/Face_Mask_data')

! kaggle datasets download -d andrewmvd/face-mask-detection --force


 99% 394M/398M [00:11<00:00, 54.6MB/s]
100% 398M/398M [00:11<00:00, 37.0MB/s]


In [ ]:
! unzip face-mask-detection.zip

Archive:  face-mask-detection.zip
  inflating: annotations/maksssksksss0.xml  
  inflating: annotations/maksssksksss1.xml  
  inflating: annotations/maksssksksss10.xml  
  inflating: annotations/maksssksksss100.xml  
  inflating: annotations/maksssksksss101.xml  
  inflating: annotations/maksssksksss102.xml  
  inflating: annotations/maksssksksss103.xml  
  inflating: annotations/maksssksksss104.xml  
  inflating: annotations/maksssksksss105.xml  
  inflating: annotations/maksssksksss106.xml  
  inflating: annotations/maksssksksss107.xml  
  inflating: annotations/maksssksksss108.xml  
  inflating: annotations/maksssksksss109.xml  
  inflating: annotations/maksssksksss11.xml  
  inflating: annotations/maksssksksss110.xml  
  inflating: annotations/maksssksksss111.xml  
  inflating: annotations/maksssksksss112.xml  
  inflating: annotations/maksssksksss113.xml  
  inflating: annotations/maksssksksss114.xml  
  inflating: annotations/maksssksksss115.xml  
  inflating: annotations/maksssk

Since Yolo Requires the annotation file in a specific format, we will run a script to convert the XML files bounding box annotation to yolo format and save the images at a directory called "yolo_data"

In [ ]:
# Converting the XML files bounding box annotation to yolo format
% cd /content/drive/My Drive/Yolo/Face_Mask_detection
! python xml_covert_to_yolo.py

/content/drive/My Drive/Yolo/Face_Mask_detection
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
read file...
re

The dataset will be splitted in Train and Val and stored at yolov4-csp/data folder as txt file, it will then read by the training script to get back the path of the images. The Train.txt and Val.txt has record the path of the train and val set.

## Change the value of anchor boxes in darknet

In [ ]:
%cd /content/drive/My Drive/Yolo/Face_Mask_detection/darknet
! cp cfg/coco.data face.data

## creating face.data file with following data
# train = ../yolov4-csp/data/train.txt 
# val = ../yolov4-csp/data/val.txt 
# classes = 3

/content/drive/My Drive/Yolo/Face_Mask_detection/darknet


In [ ]:
! echo -e 'train = ../yolov4-csp/data/train.txt\nvalid = ../yolov4-csp/data/val.txt\nclasses= 3' > face.data

In [ ]:
# Calculate the anchor boxes value
!chmod +x ./darknet
! ./darknet detector calc_anchors face.data -num_of_clusters 9 -width 416 -height 416 -showpause

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0

 num_of_clusters = 9, width = 416, height = 416 
 read labels from 682 images 
 loaded 	 image: 682 	 box: 3343
 all loaded. 

 calculating k-means++ ...

 iterations = 103 


counters_per_class = 2628, 616, 99

 avg IoU = 77.17 % 

Saving anchors to the file: anchors.txt 
anchors =   8, 14,  14, 24,  20, 35,  29, 46,  36, 66,  57, 82,  76,126, 133,127, 156,193
^C


## Changing the value of anchor boxes base on the value that calculated previously

In [ ]:
%cd ../yolov4-csp
! sed -n -e 1028p -e 1137p -e 1246p models/yolov4-csp_416.cfg
! sed -i '1028s/ 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401/9, 16,  15, 27,  23, 39,  33, 55,  46, 82,  74, 59,  73,120, 132,131, 160,198/' models/yolov4-csp_416.cfg
! sed -i '1137s/ 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401/9, 16,  15, 27,  23, 39,  33, 55,  46, 82,  74, 59,  73,120, 132,131, 160,198/' models/yolov4-csp_416.cfg
! sed -i '1246s/ 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401/9, 16,  15, 27,  23, 39,  33, 55,  46, 82,  74, 59,  73,120, 132,131, 160,198/' models/yolov4-csp_416.cfg
! sed -n -e 1028p -e 1137p -e 1246p models/yolov4-csp_416.cfg

/content/drive/My Drive/Yolo/Face_Mask_detection/yolov4-csp
anchors =9, 16,  15, 27,  23, 39,  33, 55,  46, 82,  74, 59,  73,120, 132,131, 160,198
anchors =9, 16,  15, 27,  23, 39,  33, 55,  46, 82,  74, 59,  73,120, 132,131, 160,198
anchors =9, 16,  15, 27,  23, 39,  33, 55,  46, 82,  74, 59,  73,120, 132,131, 160,198
anchors =9, 16,  15, 27,  23, 39,  33, 55,  46, 82,  74, 59,  73,120, 132,131, 160,198
anchors =9, 16,  15, 27,  23, 39,  33, 55,  46, 82,  74, 59,  73,120, 132,131, 160,198
anchors =9, 16,  15, 27,  23, 39,  33, 55,  46, 82,  74, 59,  73,120, 132,131, 160,198


## Installing dependencies that required in the original Yolov4 Training Script

In [ ]:
! pip install -U PyYAML

     |████████████████████████████████| 596 kB 5.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
! pip install git+https://github.com/thomasbrandon/mish-cuda

  Cloning https://github.com/thomasbrandon/mish-cuda to /tmp/pip-req-build-ml2rop98
  Running command git clone -q https://github.com/thomasbrandon/mish-cuda /tmp/pip-req-build-ml2rop98
  Created wheel for mish-cuda: filename=mish_cuda-0.0.3-cp37-cp37m-linux_x86_64.whl size=3140027 sha256=408f22a4dca654f6400520fa54ff893212238dcc3a610e0d89549f503033a7b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-hcgu96u5/wheels/64/5c/8c/490e301bc07dd9f47fa5ff1a8808bb859f1dc010941e347283
Successfully built mish-cuda


In [ ]:
! pip install pyyaml tqdm matplotlib scipy

## Perform Training with the Script from Yolov4 Github

In [ ]:
! python train.py --device 0 --batch-size 4 --data data/face.yaml --cfg models/yolov4-csp_416.cfg --weights './runs/train/model_face7/weights/last.pt' --name model_face --epochs 50

Using torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MB)

Namespace(adam=False, batch_size=4, bucket='', cache_images=False, cfg='models/yolov4-csp_416.cfg', data='data/face.yaml', device='0', epochs=50, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], local_rank=-1, log_imgs=16, multi_scale=False, name='model_face', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/model_face8', single_cls=False, sync_bn=False, total_batch_size=4, weights='./runs/train/model_face7/weights/last.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.3, 'cls_pw': 1.0, 'obj': 0.7, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0

Inference



In [ ]:
! mkdir inference && mkdir inference/images

mkdir: cannot create directory ‘inference’: File exists


In [3]:
%cd /content/drive/MyDrive/Yolo/Face_Mask_detection/yolov4-csp

/content/drive/MyDrive/Yolo/Face_Mask_detection/yolov4-csp


In [4]:
! python detect.py --weights runs/train/model_face8/weights/last.pt --source './inference/images' --cfg models/yolov4-csp_416.cfg --names data/face.names

Namespace(agnostic_nms=False, augment=False, cfg='models/yolov4-csp_416.cfg', classes=None, conf_thres=0.4, device='', img_size=640, iou_thres=0.5, names='data/face.names', output='inference/output', save_txt=False, source='./inference/images', update=False, view_img=False, weights=['runs/train/model_face8/weights/last.pt'])
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
image 1/5 /content/drive/My Drive/Yolo/Face_Mask_detection/yolov4-csp/inference/images/0.jpg: 640x640 1 without_masks, Done. (0.211s)
image 2/5 /content/drive/My Drive/Yolo/Face_Mask_detection/yolov4-csp/inference/images/2.png: 448x640 3 with_masks, 3 without_masks, Done. (0.145s)
image 3/5 /content/drive/My Drive/Yolo/Face_Mask_detection/yolov4-csp/inference